In [1]:
import re
import warnings

import nltk
import pandas as pd
import pymorphy3
from nltk.corpus import stopwords
from corus import load_lenta
from tqdm import tqdm

from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline

warnings.filterwarnings('ignore')
tqdm.pandas()

random_state = 42


### Загрузка и обработка данных

In [2]:
#!curl -L https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz -o lenta-ru-news.csv.gz

In [3]:
records = load_lenta('lenta-ru-news.csv.gz')
data = pd.DataFrame(records)
data.columns = ['url', 'title', 'text', 'topic', 'tags', 'date']
data = data[['title', 'text', 'topic']]
data = data.sample(n=50000, random_state=random_state)
data.head()

,title,text,topic
153198,EgyptAir объявила о подорожании билетов,Египетский перевозчик EgyptAir сообщил о возмо...,Путешествия
169154,Глава Красногорского района Подмосковья ушел в...,Глава Красногорского района Московской области...,Россия
83745,Милонов предложил запретить россиянам сидеть в...,Депутат Виталий Милонов внес в Госдуму законоп...,Россия
10029,Женщинам в детородном возрасте разрешили посещ...,Верховный суд Индии разрешил женщинам в фертил...,Мир
6445,Россиянам пообещали дешевый хлеб,Россиянам не стоит бояться роста цен на хлеб —...,Экономика


In [4]:
data['topic'].value_counts()

topic
Россия               10886
Мир                   9228
Экономика             5426
Спорт                 4322
Наука и техника       3613
Культура              3600
Бывший СССР           3559
Интернет и СМИ        3135
Из жизни              1856
Дом                   1414
Силовые структуры     1371
Ценности               514
Бизнес                 472
Путешествия            429
69-я параллель          90
Крым                    43
Культпросвет            22
                        10
Легпром                  5
Библиотека               5
Name: count, dtype: int64

In [5]:
morph = pymorphy3.MorphAnalyzer()

nltk.download('stopwords')
stop_words = stopwords.words('russian')
' '.join(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\verai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'и в во не что он на я с со как а то все она так его но да ты к у же вы за бы по только ее мне было вот от меня еще нет о из ему теперь когда даже ну вдруг ли если уже или ни быть был него до вас нибудь опять уж вам ведь там потом себя ничего ей может они тут где есть надо ней для мы тебя их чем была сам чтоб без будто чего раз тоже себе под будет ж тогда кто этот того потому этого какой совсем ним здесь этом один почти мой тем чтобы нее сейчас были куда зачем всех никогда можно при наконец два об другой хоть после над больше тот через эти нас про всего них какая много разве три эту моя впрочем хорошо свою этой перед иногда лучше чуть том нельзя такой им более всегда конечно всю между'

In [6]:
data['text']

153198    Египетский перевозчик EgyptAir сообщил о возмо...
169154    Глава Красногорского района Московской области...
83745     Депутат Виталий Милонов внес в Госдуму законоп...
10029     Верховный суд Индии разрешил женщинам в фертил...
6445      Россиянам не стоит бояться роста цен на хлеб —...
                                ...                        
130987    На сайте WikiLeaks опубликован первый сборник ...
470451    Суд Пряжинского района Карелии постановил прио...
219814    В России сформировалось «поколение Путина». К ...
1428      Минздрав Саратовской области проверит сообщени...
557783    Британская актриса Хелен Миррен, сыгравшая Ели...
Name: text, Length: 50000, dtype: object

*Предобработка текстов*

Реализуем функцию предобработки текстов. В данной функции:
- Приводим текст к нижнему регистру.
- Удаляем HTML-теги.
- Оставляем только символы русского и английского алфавитов и пробелы.
- Производим токенизацию и удаляем стоп-слова.
- Выполняем лемматизацию с помощью pymorphy3.

Базовая классическая предобработка, помогает унифицировать представление текста, снизить размерность пространства признаков за счет сведения различных форм слова к единой норме и уменьшить шум. Остановилась на ней, так как лемматизация может давать потенциально лучше результат, остальные методы довольно базовые, не занимают много времени на обработку.

In [7]:
def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление HTML-тегов
    text = re.sub(r'<.*?>', ' ', text)
    # Удаление пунктуации и символов
    text = re.sub(r'[^a-zа-яё\s]', ' ', text)
    # Токенизация и удаление стоп-слов
    tokens = [word for word in text.split() if word not in stop_words]
    # Лемматизация: получение нормальной формы для каждого слова
    tokens = [morph.parse(word)[0].normal_form for word in tokens]
    return " ".join(tokens)


In [8]:
data['combined_text'] = (data['title'] + " " + data['text']).progress_apply(preprocess_text)

100%|██████████| 50000/50000 [05:13<00:00, 159.54it/s]


In [9]:
data['combined_text']

153198    egyptair объявить подорожание билет египетский...
169154    глава красногорский район подмосковье уйти отс...
83745     милон предложить запретить россиянин сидеть со...
10029     женщина детородный возраст разрешить посещать ...
6445      россиянин пообещать дешёвый хлеб россиянин сто...
                                ...                        
130987    wikileaks опубликовать электронный письмо прав...
470451    карельский пенсионер переселить пожароопасный ...
219814    политолог наслать россия поколение путин росси...
1428      чиновник заинтересоваться червь тарелка пациен...
557783    хелен миррен стать королева премия оскар брита...
Name: combined_text, Length: 50000, dtype: object

In [10]:
X = data['combined_text']
y = data['topic']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.6, stratify=y, random_state=random_state)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=random_state)

### Dummy-бейзлайн
Создаем dummy-классификатор с стратегией предсказания самого частого класса для получения базового уровня качества.

In [11]:
pipeline_dummy = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('dummy', DummyClassifier(strategy='most_frequent', random_state=random_state))
])

pipeline_dummy.fit(X_train, y_train)
y_pred_dummy = pipeline_dummy.predict(X_val)
print("Classification Report for Dummy Classifier:\n")
print(classification_report(y_val, y_pred_dummy, zero_division=0))

Classification Report for Dummy Classifier:

                   precision    recall  f1-score   support

                        0.00      0.00      0.00         2
   69-я параллель       0.00      0.00      0.00        18
       Библиотека       0.00      0.00      0.00         1
           Бизнес       0.00      0.00      0.00        95
      Бывший СССР       0.00      0.00      0.00       712
              Дом       0.00      0.00      0.00       283
         Из жизни       0.00      0.00      0.00       371
   Интернет и СМИ       0.00      0.00      0.00       627
             Крым       0.00      0.00      0.00         9
    Культпросвет        0.00      0.00      0.00         4
         Культура       0.00      0.00      0.00       720
          Легпром       0.00      0.00      0.00         1
              Мир       0.00      0.00      0.00      1845
  Наука и техника       0.00      0.00      0.00       722
      Путешествия       0.00      0.00      0.00        86
          

### Обучение модели LogisticRegression с двумя вариантами векторизации

In [12]:
# Пайплайн с CountVectorizer
pipeline_count = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('clf', LogisticRegression(random_state=random_state, max_iter=1000))
])
pipeline_count.fit(X_train, y_train)
y_pred_count = pipeline_count.predict(X_val)
print("Classification Report for CountVectorizer + LogReg:\n")
print(classification_report(y_val, y_pred_count, zero_division=0))

Classification Report for CountVectorizer + LogReg:

                   precision    recall  f1-score   support

                        0.00      0.00      0.00         2
   69-я параллель       0.75      0.17      0.27        18
       Библиотека       0.00      0.00      0.00         1
           Бизнес       0.52      0.34      0.41        95
      Бывший СССР       0.80      0.78      0.79       712
              Дом       0.86      0.83      0.85       283
         Из жизни       0.61      0.57      0.59       371
   Интернет и СМИ       0.73      0.72      0.73       627
             Крым       1.00      0.22      0.36         9
    Культпросвет        0.00      0.00      0.00         4
         Культура       0.88      0.89      0.88       720
          Легпром       0.00      0.00      0.00         1
              Мир       0.78      0.78      0.78      1845
  Наука и техника       0.81      0.82      0.82       722
      Путешествия       0.66      0.49      0.56        86
  

In [13]:
# Пайплайн с TfidfVectorizer
pipeline_tfidf = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('clf', LogisticRegression(random_state=random_state, max_iter=1000))
])
pipeline_tfidf.fit(X_train, y_train)
y_pred_tfidf = pipeline_tfidf.predict(X_val)
print("Classification Report for CountVectorizer + LogReg:\n")
print(classification_report(y_val, y_pred_tfidf, zero_division=0))

Classification Report for CountVectorizer + LogReg:

                   precision    recall  f1-score   support

                        0.00      0.00      0.00         2
   69-я параллель       0.00      0.00      0.00        18
       Библиотека       0.00      0.00      0.00         1
           Бизнес       0.80      0.08      0.15        95
      Бывший СССР       0.81      0.76      0.79       712
              Дом       0.87      0.76      0.81       283
         Из жизни       0.71      0.51      0.60       371
   Интернет и СМИ       0.78      0.70      0.73       627
             Крым       0.00      0.00      0.00         9
    Культпросвет        0.00      0.00      0.00         4
         Культура       0.86      0.88      0.87       720
          Легпром       0.00      0.00      0.00         1
              Мир       0.76      0.84      0.80      1845
  Наука и техника       0.82      0.86      0.84       722
      Путешествия       0.76      0.30      0.43        86
  

### Оптимизация гиперпараметров

Проводим GridSearchCV для одновременной оптимизации гиперпараметров векторизаторов и модели LogisticRegression.
Для LogisticRegression подбираем:
- Параметр регуляризации C.
- Тип штрафа (penalty)
- solver

Для векторизаторов подбираем параметры max_df, min_df

In [14]:
cv_settings = {
    'cv': 5,
    'scoring': 'accuracy',
    'n_jobs': -1,
    'verbose': 2
}

#### CountVectorizer

In [15]:
pipeline_count = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('clf', LogisticRegression(random_state=random_state, max_iter=1000))
])


param_grid_count = [
    {   # для solver liblinear — разрешаем l1 и l2
        'vectorizer__max_df': [0.7, 0.8, 0.9],
        'vectorizer__min_df': [0.003, 0.005, 0.01],
        'clf__C': [0.5, 1],
        'clf__penalty': ['l1', 'l2'],
        'clf__solver': ['liblinear']
    },
    {   # для solver lbfgs — только l2
        'vectorizer__max_df': [0.7, 0.8, 0.9],
        'vectorizer__min_df': [0.003, 0.005, 0.01],
        'clf__C': [0.5, 1],
        'clf__penalty': ['l2'],
        'clf__solver': ['lbfgs']
    }
]

grid_search_count = GridSearchCV(pipeline_count, param_grid_count, **cv_settings)
grid_search_count.fit(X_train, y_train)

print("=== CountVectorizer Best Parameters ===")
print(grid_search_count.best_params_)
print("Best cross-val accuracy:", grid_search_count.best_score_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
=== CountVectorizer Best Parameters ===
{'clf__C': 0.5, 'clf__penalty': 'l1', 'clf__solver': 'liblinear', 'vectorizer__max_df': 0.7, 'vectorizer__min_df': 0.003}
Best cross-val accuracy: 0.7643666666666667


In [16]:
best_model_count = grid_search_count.best_estimator_
y_val_pred_count = best_model_count.predict(X_val)
print("=== Classification Report on Validation Set (CountVectorizer) ===")
print(classification_report(y_val, y_val_pred_count, zero_division=0))

=== Classification Report on Validation Set (CountVectorizer) ===
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         2
   69-я параллель       0.25      0.11      0.15        18
       Библиотека       0.00      0.00      0.00         1
           Бизнес       0.55      0.25      0.35        95
      Бывший СССР       0.77      0.75      0.76       712
              Дом       0.82      0.78      0.80       283
         Из жизни       0.60      0.52      0.56       371
   Интернет и СМИ       0.72      0.70      0.71       627
             Крым       0.50      0.11      0.18         9
    Культпросвет        0.00      0.00      0.00         4
         Культура       0.86      0.86      0.86       720
          Легпром       0.00      0.00      0.00         1
              Мир       0.77      0.78      0.77      1845
  Наука и техника       0.80      0.80      0.80       722
      Путешествия       0.68      0.48      0.56

#### TF-IDF

In [17]:
pipeline_tfidf = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('clf', LogisticRegression(random_state=random_state, max_iter=1000))
])

param_grid_tfidf = [
    {   # для solver liblinear — l1 и l2
        'vectorizer__max_df': [0.7, 0.8, 0.9],
        'vectorizer__min_df': [0.003, 0.005, 0.01],
        'clf__C': [0.5, 1],
        'clf__penalty': ['l1', 'l2'],
        'clf__solver': ['liblinear']
    },
    {   # для solver lbfgs — только l2
        'vectorizer__max_df': [0.7, 0.8, 0.9],
        'vectorizer__min_df': [0.003, 0.005, 0.01],
        'clf__C': [0.5, 1],
        'clf__penalty': ['l2'],
        'clf__solver': ['lbfgs']
    }
]

grid_search_tfidf = GridSearchCV(pipeline_tfidf, param_grid_tfidf, **cv_settings)
grid_search_tfidf.fit(X_train, y_train)

print("=== TfidfVectorizer Best Parameters ===")
print(grid_search_tfidf.best_params_)
print("Best cross-val accuracy:", grid_search_tfidf.best_score_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
=== TfidfVectorizer Best Parameters ===
{'clf__C': 1, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs', 'vectorizer__max_df': 0.7, 'vectorizer__min_df': 0.003}
Best cross-val accuracy: 0.7758666666666667


In [18]:
best_model_tfidf = grid_search_tfidf.best_estimator_
y_val_pred_tfidf = best_model_tfidf.predict(X_val)
print("=== Classification Report on Validation Set (TfidfVectorizer) ===")
print(classification_report(y_val, y_val_pred_tfidf, zero_division=0))

=== Classification Report on Validation Set (TfidfVectorizer) ===
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         2
   69-я параллель       0.00      0.00      0.00        18
       Библиотека       0.00      0.00      0.00         1
           Бизнес       0.77      0.11      0.19        95
      Бывший СССР       0.79      0.74      0.76       712
              Дом       0.87      0.78      0.82       283
         Из жизни       0.67      0.53      0.59       371
   Интернет и СМИ       0.75      0.71      0.73       627
             Крым       0.00      0.00      0.00         9
    Культпросвет        0.00      0.00      0.00         4
         Культура       0.87      0.87      0.87       720
          Легпром       0.00      0.00      0.00         1
              Мир       0.78      0.83      0.80      1845
  Наука и техника       0.81      0.84      0.83       722
      Путешествия       0.72      0.40      0.51

### Оценка на отложенной выборке

In [20]:
# tfidf
y_test_pred_tfidf = best_model_tfidf.predict(X_test)
print("=== Classification Report on Test Set (TfidfVectorizer) ===")
print(classification_report(y_test, y_test_pred_tfidf, zero_division=0))

=== Classification Report on Test Set (TfidfVectorizer) ===
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         2
   69-я параллель       0.00      0.00      0.00        18
       Библиотека       0.00      0.00      0.00         1
           Бизнес       0.88      0.07      0.14        94
      Бывший СССР       0.79      0.79      0.79       712
              Дом       0.82      0.76      0.79       283
         Из жизни       0.69      0.54      0.60       371
   Интернет и СМИ       0.73      0.66      0.69       627
             Крым       0.00      0.00      0.00         8
    Культпросвет        0.00      0.00      0.00         5
         Культура       0.85      0.85      0.85       720
          Легпром       0.00      0.00      0.00         1
              Мир       0.76      0.84      0.80      1846
  Наука и техника       0.80      0.83      0.82       723
      Путешествия       0.71      0.47      0.56      

Dummy baseline: accuracy - 0.22

Лучшая модель - TF-IDF:
- {'clf__C': 1, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs', 'vectorizer__max_df': 0.7, 'vectorizer__min_df': 0.003}
- Best cross-val accuracy: 0.7758666666666667
- val_accuracy: 0.79
- test_accurcy: 0.79

